# Lesson I

## Comparing Strings

We'll discover the world of record linkage. But before we get deep dive into record linkage, let's sharpen our understanding of string similarity and minimum edit distance.

### Minimum edit distance

*Minimum edit distance* is a systematic way to identify how close 2 strings are. 

For example, let's take a look at the following two words: **intention**, and **execution**. The minimum edit distance between them is the least possible amount of steps, that could get us from the word intention to execution, with the available operations being:

* Insertion
* Deletion
* Substitution
* Transposition

To get from **intention** to **execution** ;

* Deleting "I" from intention
* Adding "C" between "E" and "N"
* Substitute the first "N" with "E", 
* "T" with "X" and
* "N" with "U"

***Minimum edit distance being 5!***

The lower the edit distance, the closer two words are. For example, the two different typos of **reading** have a minimum edit distance of 1 between **reeding** and **reading**.

#### Minimum edit distance Algorithms

There's a variety of algorithms based on edit distance that differ on which operations they use, how much weight attributed to each operation, which type of strings they're suited for and more, with a variety of packages to get each similarity.

| Algorithm | Operations|
|-----------|-----------|
| Damerau-Levenshtein | insertion, substitution, deletion, transposition |
| **Levenshtein** | **insertion, substitution, deletion** |
| Hamming | substitution only |
| Jaro distance | transposition only |
| ... | ... |

**Possible packages :**
* ``nltk``
* ``fuzzywuzzy``
* ``textdistance``

For this lesson, we'll be comparing strings using *Levenshtein* distance since it's the most general form of string matching by using the **fuzzywuzzy** package.

### Simple String Comparison

``Fuzzywuzzy`` is a simple to use package to perform string comparison. 
We first ``import fuzz from fuzzywuzzy``, which allow us to compare between single strings. Here we use *fuzz*'s ``WRatio()`` function to compute the similarity between reading and its typo, inputting each string as an argument. 
For any comparison function using ``fuzzywuzzy``, our output is a score from **0** to **100** with 0 being not similar at all, 100 being an exact match. 

Do not confuse this with the minimum edit distance score earlier, where a lower minimum edit distance means a closer match.

In [5]:
# Let us compare between two strings
from fuzzywuzzy import fuzz

# Compare reeding vs reading
fuzz.WRatio('Reeding', 'Reading')

86

#### Partial strings and different orderings

The ``WRatio()`` function is highly robust against partial string comparison with different orderings. 
For example here we compare the strings *Houston Rockets and Rockets*, and still receive a high similarity score.

In [6]:
# Partial string comparison
fuzz.WRatio('Houston Rockets', 'Rockets')

90

The same can be said for the strings *Houston Rockets vs Los Angeles Lakers and Lakers vs Rockets*, where the team names are only partial and they are differently ordered.

In [7]:
# Partial string comparison with different order
fuzz.WRatio('Houston Rockets vs Los Angeles Lakers', 'Lakers vs Rockets')

86

#### Comparision with arrays

We can also compare a string with an *array* of strings by using the ``extract()`` function from the ``process module from fuzzy wuzzy``. 
``extract()`` takes in a *string*, *an array of strings*, and *the number of possible matches to return ranked from highest to lowest*. 
It *returns* a **list of tuples with 3 elements**; 
* the first one being the matching string being returned, 
* the second one being its similarity score, 
* and the third one being its index in the array.

In [8]:
# Import process
from fuzzywuzzy import process
import pandas as pd

# Define string and array of possible matches
string = "Houston Rockets vs Los Angeles Lakers"
choices = pd.Series(["Rockets vs Lakers", "Lakers vs Rockets",
                     "Houston vs Los Angeles", "Heat vs Bulls"])

process.extract(string, choices, limit=2)

[('Rockets vs Lakers', 86, 0), ('Lakers vs Rockets', 86, 1)]

### Collapsing Categories with string similarity

In chapter 2, we learned that collapsing data into categories is an essential aspect of working with categorical and text data, and we saw how to manually replace categories in a column of a DataFrame. But what if we had so many inconsistent categories that a manual replacement is simply not feasible? We can easily do that with string similarity!

* Use ``.replace()`` to collapse ``"eur"`` into ``"Europe"``

* What if there are too many variations?
    - ``"EU", "eur", "Europ", "Europa", "Erope", "Evropa"`` ...

Say we have DataFrame named ``survey`` containing answers from respondents from the state of New York and California asking them how likely are you to move on a scale of 0 to 5. 

```python
print(survey['state'].unique())
```

<img src='pictures/survey.jpg' width=250 />

The state field was free text and contains hundreds of typos. Remapping them manually would take a huge amount of time. Instead, we'll use string similarity. 

We also have a ``category`` DataFrame containing the correct categories for each state(``'California'`` ``'New York'``). Let's collapse the incorrect categories with string matching!

#### Collapsing all of the state

We first create a *for loop* iterating over each correctly typed state in the ``categories`` DataFrame. 

For each state, we find its matches in the ``state`` column of the ``survey`` DataFrame, returning all possible matches by setting the ``limit`` argument of extract to the length of the ``survey`` DataFrame.

Then we iterate over each potential match, isolating the ones only with a similarity score higher or equal than 80 with an if statement. 

Then for each of those returned strings, we replace it with the correct state using the .``loc`` method.

```python
# For Each correct category
for state in categories['state']:
    # Find potential matches in states with typoes
    matches = process.extract(state, survey['state'], limit = survey.shape[0])
    # For each potential match match
    for potential_match in matches:
        # If high similarity score
        if potential_match[1] >= 80:
            # Replace typo with correct category
            survey.loc[survey['state'] == potential_match[0], 'state'] = state
```

### Record Linkage

<img src='pictures/recordlinkage.jpg' />

Record linkage attempts to join data sources that have similarly fuzzy duplicate values, so that we end up with a final DataFrame with no duplicates by using string similarity. We'll cover record linkage in more detail in the next couple of lessons.

## Exercise 

### The cutoff point

In this exercise, and throughout this chapter, you'll be working with the ``restaurants`` DataFrame which has data on various restaurants. Your ultimate goal is to create a restaurant recommendation engine, but you need to first clean your data.

This version of restaurants has been collected from many sources, where the ``cuisine_type`` column is riddled with typos, and should contain only *italian, american and asian* cuisine types. There are so many unique categories that remapping them manually isn't scalable, and it's best to use string similarity instead.

Before doing so, you want to establish the cutoff point for the similarity score using the *fuzzywuzzy*'s ``process.extract()`` function by finding the similarity score of the most distant typo of each category.